# FASE 1: Pipeline ETL - Extrator de Simulados (PDF → JSON)
**Autor:** Amaro Netto

**Objetivo:** Automatizar a extração de dados não estruturados (PDF) para formato estruturado (JSON), preparando o terreno para enriquecimento com IA.

---

## 📦 1. Instalação de Dependências
Instalação silenciosa das ferramentas de OCR (`pdfplumber`), manipulação de dados e visualização de progresso (`tqdm`, `ipywidgets`).

In [ ]:
%%capture
!pip install pdfplumber tqdm ipywidgets pandas

In [33]:
import sys

# Verifica se houve erro na saída capturada
if "ERROR:" in str(captured_output):
    print("❌ Ocorreu um erro na instalação:")
    captured_output.show()
else:
    print("✅ Todas as bibliotecas foram instaladas e estão prontas para uso.")

✅ Todas as bibliotecas foram instaladas e estão prontas para uso.


## ⚙️ 2. Configuração de Diretórios
Definição dos caminhos de entrada e saída. O script cria automaticamente as pastas `txt` e `json` caso não existam, garantindo que o pipeline nunca falhe por erro de diretório.

In [34]:
import pdfplumber
import re
import json
import os
from pathlib import Path

# Configuração de Pastas (Cria automático se não existir)
PASTA_PDF = Path("pdf")     # Onde você joga os arquivos originais
PASTA_TXT = Path("txt")     # Onde fica o texto intermediário
PASTA_JSON = Path("json")   # Onde sai o resultado final

PASTA_TXT.mkdir(exist_ok=True)
PASTA_JSON.mkdir(exist_ok=True)

print("✅ Ambiente configurado. Pastas prontas.")

✅ Ambiente configurado. Pastas prontas.


## 📄 3. Módulo de Extração (PDF Reader)
Função robusta que converte o binário PDF em fluxo de texto contínuo.
* **Técnica:** Utilizamos `layout=False` para priorizar a ordem de leitura humana.
* **UX:** Integrada com a barra de progresso para mostrar o avanço das páginas em tempo real.

In [35]:
def pdf_para_txt_stream(caminho_pdf):
    texto_completo = []
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            for page in pdf.pages:
                # layout=False ignora posições exatas e pega o fluxo de texto (melhor para sua prova)
                txt = page.extract_text(layout=False)
                if txt:
                    texto_completo.append(txt)
        return "\n".join(texto_completo)
    except Exception as e:
        print(f"❌ Erro no arquivo {caminho_pdf}: {e}")
        return None

## 🛠️ 4. Módulo de Transformação (Business Logic)
Onde a mágica acontece. O texto bruto é limpo, segmentado e estruturado.
1.  **Gabarito:** Identificado via Regex no rodapé do arquivo.
2.  **Questões:** Separadas pelo padrão `Questão X`.
3.  **Alternativas:** Identificadas e mapeadas para garantir a ordem correta (A, B, C, D, E).

In [36]:
def processar_texto_para_json(texto_completo, id_inicial=1):
    # 1. Extrair Gabarito
    gabarito_map = {}
    match_inicio = re.search(r'Respostas:', texto_completo, re.IGNORECASE)
    texto_fim = texto_completo[match_inicio.start():] if match_inicio else texto_completo[-5000:]
    
    matches_gab = re.findall(r'(\d+)\s+([A-E])', texto_fim)
    for num, letra in matches_gab:
        gabarito_map[int(num)] = letra

    # 2. Separar Questões
    divisao_gabarito = re.split(r'Respostas:', texto_completo, flags=re.IGNORECASE)
    texto_questoes = divisao_gabarito[0] if divisao_gabarito else texto_completo
    blocos = re.split(r'(Questão\s+\d+)', texto_questoes, flags=re.IGNORECASE)

    lista_final = []
    
    for i in range(1, len(blocos)-1, 2):
        cabecalho = blocos[i]
        conteudo = blocos[i+1]
        
        # ID Seguro
        match_num = re.search(r'\d+', cabecalho)
        if not match_num: continue
        id_q = int(match_num.group())

        # Processar Linhas (Desembaralhar alternativas)
        enunciado_parts = []
        opcoes_map = {}
        
        for linha in conteudo.split('\n'):
            # Limpeza rápida
            linha = linha.strip()
            if not linha or re.match(r'^\d{8,}$', linha) or "Acessar Lista" in linha: continue
            
            # Captura Alternativa (A, B, C...)
            match_alt = re.match(r'^([A-E])[\s\.\)\-](.*)', linha)
            if match_alt:
                opcoes_map[match_alt.group(1)] = match_alt.group(2).strip()
            elif len(linha) > 3:
                enunciado_parts.append(linha)

        # Montagem
        opcoes_lista = [opcoes_map.get(letra, "N/A") for letra in ['A','B','C','D','E']]
        letra_resp = gabarito_map.get(id_q)

        lista_final.append({
            "id": id_q,
            "text": " ".join(enunciado_parts),
            "options": opcoes_lista,
            "correct": letra_resp, # Sai "A", "B", etc.
            "explanation": ""
        })
        
    return lista_final

## 🚀 5. Execução do Pipeline
Orquestrador que processa todos os arquivos da pasta `pdf/`.
Fornecer feedback visual do status do processamento.

In [37]:
# 1. Listar PDFs
arquivos = list(PASTA_PDF.glob("*.pdf"))
print(f"🚀 Iniciando Pipeline para {len(arquivos)} arquivos PDF...\n")

for arq_pdf in arquivos:
    print(f"📄 Processando: {arq_pdf.name}")
    
    # ETAPA 1: PDF -> TXT
    texto_cru = pdf_para_txt_stream(arq_pdf)
    
    if texto_cru:
        # Salva backup TXT (Opcional, mas boa prática)
        caminho_txt = PASTA_TXT / (arq_pdf.stem + ".txt")
        with open(caminho_txt, "w", encoding="utf-8") as f:
            f.write(texto_cru)
            
        # ETAPA 2: TXT -> JSON
        dados_json = processar_texto_para_json(texto_cru)
        
        # Salva JSON Final
        caminho_json = PASTA_JSON / (arq_pdf.stem + ".json")
        with open(caminho_json, "w", encoding="utf-8") as f:
            json.dump(dados_json, f, indent=2, ensure_ascii=False)
            
        print(f"   ✅ Gerado JSON com {len(dados_json)} questões.")
    else:
        print("   ❌ Falha na leitura do PDF.")

print("\n🏁 Pipeline Finalizado!")

🚀 Iniciando Pipeline para 12 arquivos PDF...

📄 Processando: ALERJ 2025 Arquivologia - FGV.pdf
   ✅ Gerado JSON com 87 questões.
📄 Processando: ALERJ 2025 Arquivologia Diversas Bancas.pdf
   ✅ Gerado JSON com 293 questões.
📄 Processando: ALERJ 2025 Legislação Específica Diversas Bancas.pdf
   ✅ Gerado JSON com 66 questões.
📄 Processando: ALERJ 2025 Língua Portuguesa - FGV.pdf
   ✅ Gerado JSON com 855 questões.
📄 Processando: ALERJ 2025 Noções de Administração - Diversas Bancas.pdf
   ✅ Gerado JSON com 971 questões.
📄 Processando: ALERJ 2025 Noções de Administração - FGV.pdf
   ✅ Gerado JSON com 298 questões.
📄 Processando: ALERJ 2025 Noções de Direito Administrativo - FGV.pdf
   ✅ Gerado JSON com 445 questões.
📄 Processando: ALERJ 2025 Noções de Direito Constitucional - Diversas Bancas.pdf
   ✅ Gerado JSON com 710 questões.
📄 Processando: ALERJ 2025 Noções de Direito Constitucional - FGV.pdf
   ✅ Gerado JSON com 489 questões.
📄 Processando: ALERJ 2025 Noções de Informática - Diversas B

## 📊 6. Validação e Auditoria (Quality Assurance)
Gera um relatório consolidado de todos os arquivos processados, verificando:
1.  **Volumetria:** Quantidade total de questões extraídas por arquivo.
2.  **Integridade:** Identificação de questões sem gabarito (erros de leitura).
3.  **Amostragem:** Visualização aleatória de questões de diferentes arquivos para conferência humana.

In [42]:
import pandas as pd
import json

# Lista todos os JSONs gerados
arquivos_json = list(PASTA_JSON.glob("*.json"))

if not arquivos_json:
    print("⚠️ Nenhum arquivo JSON encontrado para auditar.")
else:
    # Listas para armazenar dados consolidados
    resumo_stats = []
    todas_questoes = []

    print(f"🔍 Iniciando auditoria em {len(arquivos_json)} arquivos...\n")

    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
            
            # 1. Coleta Estatísticas
            total_q = len(dados)
            # Conta quantas questões estão sem gabarito (correct = None)
            sem_gabarito = sum(1 for q in dados if q['correct'] is None)
            
            resumo_stats.append({
                "Nome do Arquivo": arquivo.name,
                "Qtd Questões": total_q,
                "⚠️ Sem Gabarito": sem_gabarito, # Útil para debug
                "Status": "✅ OK" if total_q > 0 else "❌ VAZIO"
            })
            
            # 2. Adiciona ao 'Lakedatal' (adicionando a origem para sabermos de onde veio)
            for q in dados:
                q['origem'] = arquivo.name # Adiciona metadado
                todas_questoes.append(q)
                
        except Exception as e:
            resumo_stats.append({
                "Nome do Arquivo": arquivo.name,
                "Qtd Questões": 0,
                "Status": f"❌ ERRO: {str(e)}"
            })

    # --- EXIBIÇÃO DOS RESULTADOS ---

    # 1. Tabela Resumo por Arquivo
    df_resumo = pd.DataFrame(resumo_stats)
    print("📋 TABELA DE VOLUMETRIA:")
    # Estiliza para destacar erros (opcional, se der erro tire o .style...)
    display(df_resumo)

    # 2. Totais Gerais
    total_geral = len(todas_questoes)
    print(f"\n🌎 TOTAL GLOBAL: {total_geral} questões extraídas.")

    # 3. Amostra Aleatória (Sampling)
    if total_geral > 0:
        df_completo = pd.DataFrame(todas_questoes)
        print("\n🎲 AMOSTRA ALEATÓRIA (5 Questões variadas para conferência):")
        # Pega 5 questões aleatórias para você ler e ver se o texto faz sentido
        amostra = df_completo[['origem', 'id', 'text', 'correct']].sample(n=min(5, total_geral))
        pd.set_option('display.max_colwidth', 100) # Aumenta largura para ler o texto
        display(amostra)
    else:
        print("⚠️ Nenhuma questão válida encontrada no total.")

🔍 Iniciando auditoria em 12 arquivos...

📋 TABELA DE VOLUMETRIA:


,Nome do Arquivo,Qtd Questões,⚠️ Sem Gabarito,Status
0,ALERJ 2025 Arquivologia - FGV.json,87,0,✅ OK
1,ALERJ 2025 Arquivologia Diversas Bancas.json,293,0,✅ OK
2,ALERJ 2025 Legislação Específica Diversas Banc...,66,0,✅ OK
3,ALERJ 2025 Língua Portuguesa - FGV.json,855,0,✅ OK
4,ALERJ 2025 Noções de Administração - Diversas ...,971,0,✅ OK
5,ALERJ 2025 Noções de Administração - FGV.json,298,0,✅ OK
6,ALERJ 2025 Noções de Direito Administrativo - ...,445,0,✅ OK
7,ALERJ 2025 Noções de Direito Constitucional - ...,710,0,✅ OK
8,ALERJ 2025 Noções de Direito Constitucional - ...,489,0,✅ OK
9,ALERJ 2025 Noções de Informática - Diversas Ba...,820,0,✅ OK



🌎 TOTAL GLOBAL: 6239 questões extraídas.

🎲 AMOSTRA ALEATÓRIA (5 Questões variadas para conferência):


,origem,id,text,correct
5337,ALERJ 2025 Noções de Informática - Vunesp.json,304,"Lista de Atalhos Um funcionário da ALBA digitou um trabalho no Word 2010 BR para Windows e, ao (...",B
515,ALERJ 2025 Língua Portuguesa - FGV.json,70,"Emprego do hífen Notamos, como usuários da língua portuguesa, que alguns vocábulos podem ser usa...",E
3440,ALERJ 2025 Noções de Direito Constitucional - Diversas Bancas.json,426,Da responsabilidade do Presidente da República I II Admitida acusação contra o Presidente da Rep...,D
4430,ALERJ 2025 Noções de Informática - Diversas Bancas.json,217,"Conceitos Básicos O programa MS-Excel 2010, em sua con(cid:39)guração padrão, está sendo utiliza...",A
4698,ALERJ 2025 Noções de Informática - Diversas Bancas.json,485,"Novidades Word 2013 Guia Inserir No Microsoft Word 2013, para juntar alguns trechos de um determ...",A


# Fase 2: Enriquecimento e Persistência

## 📦 7. Configuração da Fase 2
1.  Instalação das bibliotecas necessárias para conectar com a Inteligência Artificial (Google Gemini) e gerenciar variáveis de ambiente de forma segura.
2. Passo 2: Atualizar a Biblioteca
3. Passo 3: Descobrir o Nome Correto do Modelo que será utilizado.

In [39]:
%%capture
!pip install google-generativeai python-dotenv

In [43]:
%%capture
!pip install --upgrade google-generativeai

In [44]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=API_KEY)

print("🔍 Buscando modelos disponíveis...")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"✅ Disponível: {m.name}")
except Exception as e:
    print(f"❌ Erro ao listar: {e}")

🔍 Buscando modelos disponíveis...
✅ Disponível: models/gemini-2.5-flash
✅ Disponível: models/gemini-2.5-pro
✅ Disponível: models/gemini-2.0-flash-exp
✅ Disponível: models/gemini-2.0-flash
✅ Disponível: models/gemini-2.0-flash-001
✅ Disponível: models/gemini-2.0-flash-exp-image-generation
✅ Disponível: models/gemini-2.0-flash-lite-001
✅ Disponível: models/gemini-2.0-flash-lite
✅ Disponível: models/gemini-2.0-flash-lite-preview-02-05
✅ Disponível: models/gemini-2.0-flash-lite-preview
✅ Disponível: models/gemini-exp-1206
✅ Disponível: models/gemini-2.5-flash-preview-tts
✅ Disponível: models/gemini-2.5-pro-preview-tts
✅ Disponível: models/gemma-3-1b-it
✅ Disponível: models/gemma-3-4b-it
✅ Disponível: models/gemma-3-12b-it
✅ Disponível: models/gemma-3-27b-it
✅ Disponível: models/gemma-3n-e4b-it
✅ Disponível: models/gemma-3n-e2b-it
✅ Disponível: models/gemini-flash-latest
✅ Disponível: models/gemini-flash-lite-latest
✅ Disponível: models/gemini-pro-latest
✅ Disponível: models/gemini-2.5-flas

## 🗄️ 8. Camada de Persistência (SQLite)
Nesta etapa, consolidamos os arquivos JSON dispersos em um banco de dados relacional (SQLite).
* **Idempotência:** O script verifica se a questão já existe antes de inserir, evitando duplicatas se rodarmos o pipeline múltiplas vezes.
* **Schema:** Criamos uma tabela `questoes` otimizada com flags de controle (`processado`) para gerenciar o fluxo da IA.

In [2]:
import sqlite3
import json
import os
from pathlib import Path

# Configuração
PASTA_JSON = Path("json")
DB_NAME = "simulado.db"

def criar_tabela(conn):
    cursor = conn.cursor()
    # Tabela robusta com campo de controle 'processado'
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS questoes (
            id_global INTEGER PRIMARY KEY AUTOINCREMENT,
            id_original INTEGER,
            arquivo_origem TEXT,
            enunciado TEXT,
            opcao_a TEXT,
            opcao_b TEXT,
            opcao_c TEXT,
            opcao_d TEXT,
            opcao_e TEXT,
            gabarito TEXT,
            explicacao TEXT,
            processado BOOLEAN DEFAULT 0
        )
    """)
    conn.commit()

def importar_json(conn, arquivo_json):
    cursor = conn.cursor()
    arquivo_nome = arquivo_json.name
    
    with open(arquivo_json, 'r', encoding='utf-8') as f:
        dados = json.load(f)
        
    cont = 0
    for q in dados:
        # Verifica se já existe para não duplicar (Idempotência)
        cursor.execute("SELECT 1 FROM questoes WHERE id_original = ? AND arquivo_origem = ?", 
                       (q['id'], arquivo_nome))
        if cursor.fetchone():
            continue
            
        # Insere
        cursor.execute("""
            INSERT INTO questoes (id_original, arquivo_origem, enunciado, 
                                  opcao_a, opcao_b, opcao_c, opcao_d, opcao_e, gabarito)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            q['id'], 
            arquivo_nome, 
            q['text'],
            q['options'][0], q['options'][1], q['options'][2], q['options'][3], q['options'][4],
            q['correct']
        ))
        cont += 1
    
    conn.commit()
    print(f"   -> Importadas {cont} questões de {arquivo_nome}")

def main():
    print("🗄️  INICIANDO SETUP DO BANCO DE DADOS...")
    
    conn = sqlite3.connect(DB_NAME)
    criar_tabela(conn)
    
    arquivos = list(PASTA_JSON.glob("*.json"))
    if not arquivos:
        print("❌ Nenhum JSON encontrado na pasta json/.")
        return

    for arq in arquivos:
        importar_json(conn, arq)
        
    print(f"\n✅ Banco '{DB_NAME}' pronto e populado!")
    conn.close()

if __name__ == "__main__":
    main()

🗄️  INICIANDO SETUP DO BANCO DE DADOS...
   -> Importadas 87 questões de ALERJ 2025 Arquivologia - FGV.json
   -> Importadas 293 questões de ALERJ 2025 Arquivologia Diversas Bancas.json
   -> Importadas 66 questões de ALERJ 2025 Legislação Específica Diversas Bancas.json
   -> Importadas 855 questões de ALERJ 2025 Língua Portuguesa - FGV.json
   -> Importadas 971 questões de ALERJ 2025 Noções de Administração - Diversas Bancas.json
   -> Importadas 298 questões de ALERJ 2025 Noções de Administração - FGV.json
   -> Importadas 445 questões de ALERJ 2025 Noções de Direito Administrativo - FGV.json
   -> Importadas 710 questões de ALERJ 2025 Noções de Direito Constitucional - Diversas Bancas.json
   -> Importadas 489 questões de ALERJ 2025 Noções de Direito Constitucional - FGV.json
   -> Importadas 820 questões de ALERJ 2025 Noções de Informática - Diversas Bancas.json
   -> Importadas 420 questões de ALERJ 2025 Noções de Informática - Vunesp.json
   -> Importadas 785 questões de ALERJ 2

## 🧠 9. Worker de Inteligência Artificial (Google Gemini)
Um agente autônomo que consulta o banco de dados em busca de questões sem explicação.
1.  **Fila de Trabalho:** Seleciona questões onde `explicacao IS NULL`.
2.  **Engenharia de Prompt:** Envia a questão para o modelo `gemini-1.5-flash` com instruções pedagógicas estritas.
3.  **Atualização Atômica:** Salva a explicação gerada no banco e marca a questão como processada.

In [3]:
import sqlite3
import google.generativeai as genai
import time
import os
from dotenv import load_dotenv

# Carrega a API Key
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    print("❌ ERRO: Chave GEMINI_API_KEY não encontrada.")
else:
    genai.configure(api_key=API_KEY)
    
    # --- ATUALIZAÇÃO: Usando o modelo validado da sua lista ---
    # Opções que você tem: 'gemini-2.0-flash', 'gemini-2.0-pro', 'gemini-flash-latest'
    NOME_MODELO = 'gemini-2.0-flash' 
    
    try:
        model = genai.GenerativeModel(NOME_MODELO)
        print(f"🤖 Modelo configurado e validado: {NOME_MODELO}")
    except Exception as e:
        print(f"⚠️ Erro ao configurar modelo: {e}")
        # Fallback de segurança
        NOME_MODELO = 'gemini-pro'
        model = genai.GenerativeModel(NOME_MODELO)

    DB_NAME = "simulado.db"

    def gerar_explicacao_didatica(enunciado, gabarito, opcoes):
        prompt = f"""
        Você é um professor especialista em concursos públicos.
        Tarefa: Explique de forma DIDÁTICA e SUCINTA (máx 3 frases) por que a alternativa ({gabarito}) é a correta.
        
        Questão: {enunciado}
        Alternativas:
        A) {opcoes[0]}
        B) {opcoes[1]}
        C) {opcoes[2]}
        D) {opcoes[3]}
        E) {opcoes[4]}
        
        Gabarito Oficial: {gabarito}
        """
        try:
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"   ⚠️ Erro API ({NOME_MODELO}): {e}")
            return None

    def worker_processar_fila():
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        
        print(f"🚀 INICIANDO WORKER COM {NOME_MODELO} (Ctrl+C para parar)...")
        
        # Lista de IDs que deram erro nesta sessão para não travar o loop
        ids_com_erro = set()
        
        while True:
            # Busca 100 questões pendentes para tentar processar
            cursor.execute("""
                SELECT id_global, id_original, enunciado, gabarito, opcao_a, opcao_b, opcao_c, opcao_d, opcao_e 
                FROM questoes 
                WHERE explicacao IS NULL AND gabarito IS NOT NULL
                LIMIT 100 
            """)
            tarefas = cursor.fetchall()
            
            # Encontra a primeira da lista que não deu erro hoje
            tarefa_atual = None
            for t in tarefas:
                if t[0] not in ids_com_erro:
                    tarefa_atual = t
                    break
            
            if not tarefa_atual:
                print("✅ Todas as questões pendentes foram processadas (ou ignoradas por erro).")
                break
            
            id_global, id_original, enunciado, gabarito, *opcoes = tarefa_atual
            
            print(f"⚡ Q.{id_original} (ID:{id_global}) Gab:{gabarito}...", end="", flush=True)
            
            # Chama a IA
            explicacao = gerar_explicacao_didatica(enunciado, gabarito, opcoes)
            
            if explicacao:
                cursor.execute("UPDATE questoes SET explicacao = ?, processado = 1 WHERE id_global = ?", (explicacao, id_global))
                conn.commit()
                print(" OK!")
                # O Gemini 2.0 é rápido, mas vamos manter uma pausa de segurança
                time.sleep(1.0) 
            else:
                print(" FALHOU. Pulando...")
                ids_com_erro.add(id_global)
                time.sleep(1)

        conn.close()

    # Executa o worker
    worker_processar_fila()

🤖 Modelo configurado e validado: gemini-2.0-flash
🚀 INICIANDO WORKER COM gemini-2.0-flash (Ctrl+C para parar)...
⚡ Q.1 (ID:1) Gab:B...

 OK!
⚡ Q.2 (ID:2) Gab:D... OK!
⚡ Q.3 (ID:3) Gab:E... OK!
⚡ Q.4 (ID:4) Gab:A... OK!
⚡ Q.5 (ID:5) Gab:A... OK!
⚡ Q.6 (ID:6) Gab:E... OK!
⚡ Q.7 (ID:7) Gab:D... OK!
⚡ Q.8 (ID:8) Gab:A... OK!
⚡ Q.9 (ID:9) Gab:D... OK!
⚡ Q.10 (ID:10) Gab:D... OK!
⚡ Q.11 (ID:11) Gab:C... OK!
⚡ Q.12 (ID:12) Gab:A... OK!
⚡ Q.13 (ID:13) Gab:D... OK!
⚡ Q.14 (ID:14) Gab:B... OK!
⚡ Q.15 (ID:15) Gab:C... OK!
⚡ Q.16 (ID:16) Gab:C... OK!
⚡ Q.17 (ID:17) Gab:B... OK!
⚡ Q.18 (ID:18) Gab:E... OK!
⚡ Q.19 (ID:19) Gab:D... OK!
⚡ Q.20 (ID:20) Gab:B... OK!
⚡ Q.21 (ID:21) Gab:C... OK!
⚡ Q.22 (ID:22) Gab:A... OK!
⚡ Q.23 (ID:23) Gab:E... OK!
⚡ Q.24 (ID:24) Gab:E... OK!
⚡ Q.25 (ID:25) Gab:A... OK!
⚡ Q.26 (ID:26) Gab:D... OK!
⚡ Q.27 (ID:27) Gab:A... OK!
⚡ Q.28 (ID:28) Gab:E... OK!
⚡ Q.29 (ID:29) Gab:C... OK!
⚡ Q.30 (ID:30) Gab:E... OK!
⚡ Q.31 (ID:31) Gab:C... OK!
⚡ Q.32 (ID:32) Gab:A... OK!
⚡ Q.33 (ID:33) Gab:B... OK!
⚡ Q.34 (ID:34) Gab:C... OK!
⚡ Q.35 (ID:35) Gab:E... OK!
⚡ Q.36 (ID:36) Gab:B... OK!
⚡ Q.37 (ID:37) Gab:B... OK!
⚡ Q

## 🏆 10. Resultado Final
Consulta ao banco de dados para visualizar o produto final: Questões estruturadas e enriquecidas com explicação gerada por IA.

In [5]:
import pandas as pd  # <--- Essa linha estava faltando nessa execução
import sqlite3

# Configuração do banco (caso a variável DB_NAME tenha sido perdida no restart)
DB_NAME = "simulado.db" 

conn = sqlite3.connect(DB_NAME)

print("🔍 Auditando dados enriquecidos no banco...")

# Carrega os dados enriquecidos para o Pandas
# Pegamos questões que já têm explicação (processadas)
df_final = pd.read_sql_query("""
    SELECT id_global, enunciado, gabarito, explicacao 
    FROM questoes 
    WHERE explicacao IS NOT NULL 
    LIMIT 5
""", conn)

conn.close()

if not df_final.empty:
    # Ajusta exibição para ler o texto todo (não cortar com '...')
    pd.set_option('display.max_colwidth', None)
    display(df_final)
else:
    print("⚠️ Nenhuma questão enriquecida encontrada ainda. Rode o Worker de IA (Célula anterior) um pouco mais.")

🔍 Auditando dados enriquecidos no banco...


,id_global,enunciado,gabarito,explicacao
0,1,"Introdução à Arquivologia Com a cessação de atividades de instituições públicas e de caráter público, sua documentação deve ser encaminhada",B,"A alternativa (B) está correta porque, em caso de cessação de atividades, a documentação deve ser transferida para a instituição arquivística pública competente ou para a instituição que assumir as responsabilidades da extinta, garantindo a preservação e o acesso aos documentos. As demais alternativas mencionam órgãos ou sistemas com outras funções, ou fases do ciclo de vida documental, não sendo adequadas para o destino final da documentação de uma instituição extinta."
1,2,Arquivo,D,"A alternativa correta (D) é ""relação orgânica"" porque um arquivo, diferentemente de uma simples coleção, é caracterizado pela interconexão natural e funcional de seus documentos, refletindo as atividades de uma entidade. Essa relação orgânica preserva o contexto e a integridade informacional, elementos essenciais para a pesquisa e a gestão documental. As demais alternativas focam em características que podem estar presentes, mas não definem a essência de um arquivo."
2,3,"Introdução à Arquivologia De acordo com a teoria das 3 idades, os documentos considerados de valor histórico-cultural para determinada organização, abertos ao público geral para consultas, são preservados como arquivos do tipo",E,"A alternativa (E) está correta porque, na teoria das três idades, os arquivos permanentes são aqueles com valor histórico-cultural, destinados à guarda definitiva e à consulta pública. Arquivos correntes e intermediários são fases anteriores, enquanto ""primário"" não se encaixa na terminologia arquivística. Portanto, documentos de valor histórico são, por definição, arquivos permanentes."
3,4,"Documento Informação e Suporte Quanto à Espécie e Tipo Quanto à Forma Relacione os exemplos à direita com o tipo de característica dos documentos a serem organizados em um escritório. 1. Forma 2. Formato 3. Espécie 4. Suporte ( ) rascunho, minuta, original, cópia ( ) declaração, relatório, ata ( ) papel, mídia eletrônica, meio magnético ( ) cartaz, folha, planta, mapa Essa questão possui comentário do professor no site 4001726641",A,"A alternativa (A) está correta porque relaciona adequadamente as características dos documentos aos seus exemplos: ""forma"" refere-se ao rascunho/minuta/original/cópia, ""espécie"" à declaração/relatório/ata, ""suporte"" ao papel/mídia eletrônica, e ""formato"" ao cartaz/folha/planta/mapa. A ordem 1-3-4-2 reflete essa correspondência lógica na organização documental."
4,5,"Introdução à Arquivologia memória e garantir transparência e acesso à informação. Isso inclui a organização, conservação e recuperação e(cid:67)cientes de documentos, bem como a definição de políticas para arquivos físicos e digitais. Com relação às noções de arquivologia, analise as afirmativas a seguir. I. O princípio da proveniência exige que conjuntos documentais (fundos) de origens distintas não sejam mesclados, prevendo uma articulação do organismo produtor do documento com a função que este representa. II. A teoria das três idades classi(cid:67)ca os arquivos em primários, secundários e terciários, de acordo com a frequência de uso por suas entidades produtoras e a identificação de seus valores permanentes. III. A gestão de documentos abrange o conjunto de procedimentos e de operações técnicas referentes à produção, ao uso, à avaliação e ao arquivamento de documentos em fase corrente e intermediária, visando à sua alienação. Está correto o que se afirma em",A,"A alternativa (A) está correta porque apenas a afirmativa I descreve corretamente um princípio fundamental da Arquivologia: o princípio da proveniência, que garante a integridade e o contexto dos documentos ao preservar sua origem e evitar a mistura de fundos documentais distintos. As demais afirmativas apresentam informações incorretas sobre a teoria das três idades e a gestão de documentos."


## 11📝 Célula 11: Migração do Banco de Dados (Schema Evolution)
#### Esta célula vai criar a tabela topicos e conectar tudo.

In [ ]:
import sqlite3

DB_NAME = "simulado.db"

def migrar_para_arquitetura_kb():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    print("🏗️ Migrando para Arquitetura de Base de Conhecimento...")
    
    # 1. Criar Tabela de Tópicos (Sua "Enciclopédia")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS topicos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            materia TEXT,
            nome TEXT,
            resumo_teorico TEXT,
            citacao_legal TEXT,
            UNIQUE(materia, nome) 
        )
    """)
    
    # 2. Conectar as Questões aos Tópicos
    try:
        cursor.execute("ALTER TABLE questoes ADD COLUMN topico_id INTEGER REFERENCES topicos(id)")
        print("   ✅ Coluna de ligação 'topico_id' adicionada.")
    except sqlite3.OperationalError:
        print("   ℹ️ Coluna 'topico_id' já existia.")

    # 3. Adicionar colunas individuais (caso ainda não tenha rodado a versão anterior)
    colunas_individuais = [("dificuldade", "TEXT"), ("explicacao", "TEXT")]
    for col, tipo in colunas_individuais:
        try:
            cursor.execute(f"ALTER TABLE questoes ADD COLUMN {col} {tipo}")
        except sqlite3.OperationalError:
            pass
            
    conn.commit()
    conn.close()
    print("🏁 Banco de dados pronto para a arquitetura relacional (Nível Sênior)!")

migrar_para_arquitetura_kb()

## 📝 Célula 12: O Worker Inteligente (Classificador + Gerador)
### Este é o script que faz a mágica: classifica a questão e gera a teoria apenas se o tópico for novo.

In [ ]:
import sqlite3
import google.generativeai as genai
import json
import time
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Configuração do Modelo JSON
model = genai.GenerativeModel('gemini-2.0-flash', generation_config={"response_mime_type": "application/json"})
DB_NAME = "simulado.db"

def classificar_questao(enunciado, gabarito, opcoes):
    """Passo 1: Identifica o assunto e explica a questão específica."""
    prompt = f"""
    Analise esta questão de concurso e retorne um JSON.
    Questão: {enunciado}
    Gabarito: {gabarito}
    Opções: {opcoes}
    
    JSON ESPERADO:
    {{
        "materia": "A disciplina macro (ex: Português, Direito Adm)",
        "topico_nome": "O conceito teórico chave (ex: Crase, Atos Administrativos)",
        "dificuldade": "Fácil/Média/Difícil",
        "explicacao_especifica": "Explicação breve de por que a alternativa {gabarito} está certa."
    }}
    """
    try:
        resp = model.generate_content(prompt)
        return json.loads(resp.text)
    except: return None

def gerar_teoria_topico(materia, topico):
    """Passo 2: Gera o material de apoio (SÓ CHAMADO SE O TÓPICO FOR NOVO)."""
    prompt = f"""
    Crie um material de apoio didático para estudantes.
    Matéria: {materia}
    Tópico: {topico}
    
    JSON ESPERADO:
    {{
        "resumo_teorico": "Resumo denso e didático sobre o conceito (máx 100 palavras).",
        "citacao_legal": "Artigo de lei ou norma que fundamenta (ou null)."
    }}
    """
    try:
        resp = model.generate_content(prompt)
        return json.loads(resp.text)
    except: return None

def worker_kb_inteligente():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Seleciona questões que ainda não têm tópico vinculado
    cursor.execute("SELECT id_global, enunciado, gabarito, opcao_a, opcao_b, opcao_c, opcao_d, opcao_e FROM questoes WHERE topico_id IS NULL AND gabarito IS NOT NULL")
    tarefas = cursor.fetchall()
    
    if not tarefas:
        print("✅ Todas as questões já estão classificadas e vinculadas!")
        return

    print(f"🚀 Processando {len(tarefas)} questões na Arquitetura KB...")
    barra = tqdm(total=len(tarefas))
    
    for tarefa in tarefas:
        id_q, enunc, gab, *opcoes = tarefa
        
        # 1. Classifica
        dados_q = classificar_questao(enunc, gab, opcoes)
        if not dados_q: 
            print(f"⚠️ Erro na Q. ID {id_q}"); continue
            
        materia = dados_q['materia']
        topico_nome = dados_q['topico_nome']
        
        # 2. Verifica se o tópico já existe
        cursor.execute("SELECT id FROM topicos WHERE materia = ? AND nome = ?", (materia, topico_nome))
        resultado = cursor.fetchone()
        
        topico_id = None
        
        if resultado:
            topico_id = resultado[0] # Tópico já existe, reaproveita!
        else:
            # Tópico novo -> Gera Teoria
            dados_teoria = gerar_teoria_topico(materia, topico_nome)
            if dados_teoria:
                cursor.execute("""
                    INSERT INTO topicos (materia, nome, resumo_teorico, citacao_legal)
                    VALUES (?, ?, ?, ?)
                """, (materia, topico_nome, dados_teoria['resumo_teorico'], dados_teoria['citacao_legal']))
                topico_id = cursor.lastrowid
                # print(f"   ✨ Novo Conhecimento: {topico_nome}") # Descomente se quiser ver o log
        
        # 3. Atualiza a questão
        if topico_id:
            cursor.execute("""
                UPDATE questoes 
                SET topico_id = ?, explicacao = ?, dificuldade = ?, processado = 1
                WHERE id_global = ?
            """, (topico_id, dados_q['explicacao_especifica'], dados_q['dificuldade'], id_q))
            conn.commit()
            
        barra.update(1)
        time.sleep(1.5) # Respeita a API

    conn.close()
    print("✨ Processamento finalizado!")

worker_kb_inteligente()

## 🎁 Bônus: Célula 13 (Visualizar sua Base de Conhecimento)
### Rode esta célula para ver a tabela de tópicos que você criou.

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("simulado.db")

print("📚 TOPICOS GERADOS (Base de Conhecimento):")
df_topicos = pd.read_sql_query("SELECT * FROM topicos LIMIT 10", conn)
display(df_topicos)

print("\n🔗 QUESTÕES VINCULADAS:")
df_vinculo = pd.read_sql_query("""
    SELECT q.id_global, q.enunciado, t.nome as topico_vinculado
    FROM questoes q
    JOIN topicos t ON q.topico_id = t.id
    LIMIT 5
""", conn)
display(df_vinculo)

conn.close()